<a href="https://colab.research.google.com/github/gcasaldi/CS50/blob/main/Un_benvenuto_a_Colaboratory.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [92]:
!pip install openpyxl
import pandas as pd
from datetime import datetime, timedelta
from google.colab import files
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.utils import to_categorical
from collections import Counter





In [93]:
from google.colab import files

# Carica il file Excel
uploaded = files.upload()
file_name = 'it-lotto-past-draws-archive.xls'

Saving it-superenalotto-past-draws-archive.zip to it-superenalotto-past-draws-archive (2).zip


In [112]:
# Carica il file Excel
uploaded = files.upload()
file_name = 'it-lotto-past-draws-archive.xls'
file_path = '/content/it-lotto-past-draws-archive.xls'

def carica_database(file_path):
    df = pd.read_excel(file_path, engine='xlrd', skiprows=1)
    numeri_estratti = df.select_dtypes(include='number').dropna(axis=1, how='all')
    numeri_estratti = numeri_estratti.tail(500)
    return numeri_estratti

def calcola_frequenze(df):
    frequenze = df.stack().value_counts()
    return frequenze.head(5).index.tolist()

def calcola_ritardatari(df):
    numeri_totali = set(range(1, 91))
    ultime_uscite = {}
    for numero in numeri_totali:
        try:
            ultima_pos = df.apply(lambda colonna: numero in colonna.values).iloc[::-1].idxmax()
            ultime_uscite[numero] = ultima_pos
        except:
            ultime_uscite[numero] = -1
    ritardatari = sorted(ultime_uscite, key=ultime_uscite.get, reverse=True)[:5]
    return ritardatari



Saving it-superenalotto-past-draws-archive.zip to it-superenalotto-past-draws-archive.zip


In [131]:
# Visualizza i primi 10 risultati del DataFrame
print(df.head(10))



          Data                  Giorno  Apertura   Massimo    Minimo  \
3         Data  Giorno della Settimana  0.000000  0.000000  0.000000   
4   2025-03-06                 Giovedì  0.000000  0.131148  0.230769   
5   2025-03-04                 Martedì  0.170213  0.426230  0.753846   
6   2025-03-01                  Sabato  0.000000  0.163934  0.461538   
7   2025-02-28                 Venerdì  0.063830  0.163934  0.523077   
8   2025-02-27                 Giovedì  0.595745  0.475410  0.461538   
9   2025-02-25                 Martedì  0.106383  0.918033  1.000000   
10  2025-02-22                  Sabato  0.723404  0.622951  0.692308   
11  2025-02-21                 Venerdì  0.425532  0.704918  0.692308   
12  2025-02-20                 Giovedì  0.106383  0.278689  0.369231   

     Chiusura     Volume  Col1   Col2       Col3  
3    1.000000   1.000000   6.0  JOLLY  Superstar  
4   17.898734  29.607143  36.0     20         53  
5   67.468354  77.285714  79.0     11         39  
6  

Codice per ordinare e correggere i dati:

In [146]:
import pandas as pd

# Verifica le prime righe del DataFrame per controllare la struttura
print("Prime righe del DataFrame:")
print(df.head())

# Filtro per selezionare solo le colonne pertinenti per il Superenalotto, inclusi tutti i numeri delle estrazioni
df_filtered = df[['Data', 'Giorno', '1', '2', '3', '4', '5', '6', 'JOLLY', 'Superstar']]

# Converte la colonna 'Data' in formato datetime, forzando l'errore su eventuali valori non convertibili
df_filtered['Data'] = pd.to_datetime(df_filtered['Data'], errors='coerce')

# Rimuove righe con valori NaT (Not a Time) nella colonna 'Data'
df_filtered = df_filtered.dropna(subset=['Data'])

# Ordina il DataFrame per la colonna 'Data' in ordine decrescente
df_sorted = df_filtered.sort_values(by='Data', ascending=False)

# Mostra i primi 10 risultati dopo l'ordinamento
print("\nPrimi 10 risultati ordinati:")
print(df_sorted.head(10))



Prime righe del DataFrame:
        Data   Giorno   Apertura    Massimo     Minimo   Chiusura     Volume  \
4 2025-03-06  Giovedì   1.000000  12.672131  21.538462  17.898734  29.607143   
5 2025-03-04  Martedì  16.148936  38.934426  68.092308  67.468354  77.285714   
6 2025-03-01   Sabato   1.000000  15.590164  42.076923  41.556962  39.142857   
7 2025-02-28  Venerdì   6.680851  15.590164  47.553846  62.962025  62.452381   
8 2025-02-27  Giovedì  54.021277  43.311475  42.076923  43.810127  49.738095   

   Col1 Col2 Col3  
4  36.0   20   53  
5  79.0   11   39  
6  83.0    6   67  
7  90.0   70   15  
8  72.0   34   62  


KeyError: "['1', '2', '3', '4', '5', '6', 'JOLLY', 'Superstar'] not in index"

Preprocessing dei dati

In [132]:
# Definisci le colonne numeriche (i numeri del Superenalotto)
numerical_cols = ['Apertura', 'Massimo', 'Minimo', 'Chiusura', 'Volume']

# Inizializza lo scaler per i numeri tra 1 e 90
scaler_superenalotto = MinMaxScaler(feature_range=(1, 90))

# Applica lo scaling sui dati numerici
df[numerical_cols] = scaler_superenalotto.fit_transform(df[numerical_cols])

. Preprocessing e creazione delle sequenze

In [126]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dropout, Dense

# Assumiamo che df contenga i dati già caricati

# Selezioniamo solo le colonne numeriche per l'allenamento
numerical_cols = ['Apertura', 'Massimo', 'Minimo']

# Normalizziamo i dati
scaler = MinMaxScaler(feature_range=(0, 1))
df[numerical_cols] = scaler.fit_transform(df[numerical_cols])

# Creiamo le sequenze temporali
def create_sequences(data, seq_length):
    sequences = []
    labels = []
    for i in range(len(data) - seq_length):
        sequences.append(data[i:i + seq_length])
        labels.append(data[i + seq_length])
    return np.array(sequences), np.array(labels)

# Definiamo la lunghezza delle sequenze
seq_length = 10  # Ad esempio, usando 10 giorni precedenti per prevedere il prossimo
data = df[numerical_cols].values

# Creiamo le sequenze temporali
sequences, labels = create_sequences(data, seq_length)

# Dividiamo i dati in training e validation set
X_train, X_val, y_train, y_val = train_test_split(sequences, labels, test_size=0.2, shuffle=False)

# Reshapiamo i dati per l'input nel modello LSTM
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], len(numerical_cols))
X_val = X_val.reshape(X_val.shape[0], X_val.shape[1], len(numerical_cols))


Creazione del modello LSTM

In [127]:
# Costruzione del modello LSTM
model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(seq_length, len(numerical_cols))))
model.add(Dropout(0.2))  # Regularizzazione
model.add(Dense(len(numerical_cols)))  # Prevediamo per tutte le colonne numeriche
model.compile(optimizer='adam', loss='mse')

# Allenamento del modello
model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_val, y_val))



Epoch 1/50


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step - loss: 0.2435 - val_loss: 0.2064
Epoch 2/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 334ms/step - loss: 0.2382 - val_loss: 0.2011
Epoch 3/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 143ms/step - loss: 0.2337 - val_loss: 0.1959
Epoch 4/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 97ms/step - loss: 0.2312 - val_loss: 0.1908
Epoch 5/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 95ms/step - loss: 0.2201 - val_loss: 0.1859
Epoch 6/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 136ms/step - loss: 0.2094 - val_loss: 0.1812
Epoch 7/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 108ms/step - loss: 0.2101 - val_loss: 0.1764
Epoch 8/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 124ms/step - loss: 0.1987 - val_loss: 0.1716
Epoch 9/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 141ms/step - loss: 0.1980 - val_loss: 0.1667
Epoch 10/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 91ms/step - loss: 0.1899 - val_loss: 0.1618
Epoch 11/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 139ms/step - loss: 0.1880 - val_loss: 0.1569
Epoch 12/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 145ms/step - loss: 0.1808 - val_loss: 0.1520
Epoch 

 Valutazione e previsione

In [128]:
# Valutazione del modello
loss = model.evaluate(X_val, y_val)
print(f"Validation Loss: {loss}")

# Previsione con un campione dal set di validazione
sample_index = 0  # Seleziona un indice dal set di validazione
sample = X_val[sample_index].reshape(1, seq_length, len(numerical_cols))  # Reshape per l'input nel modello
predicted_value = model.predict(sample)

# Inverse transform per riportare la previsione alla scala originale
predicted_value_original_scale = scaler.inverse_transform(predicted_value)

# Mostra la previsione
print(f"Predicted Value (Original Scale): {predicted_value_original_scale}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step - loss: 0.0451
Validation Loss: 0.04507073387503624
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 194ms/step
Predicted Value (Original Scale): [[18.19035  27.446476 32.71769 ]]


next step

In [129]:
# Previsione per i prossimi 10 passi
future_predictions = []
last_sequence = X_val[-1]  # Usa l'ultima sequenza del set di validazione

for _ in range(10):  # Prevedi i prossimi 10 passi
    prediction = model.predict(last_sequence.reshape(1, seq_length, len(numerical_cols)))
    future_predictions.append(prediction[0])  # Prevediamo per tutte le colonne
    last_sequence = np.roll(last_sequence, -1, axis=0)  # Shift della sequenza
    last_sequence[-1] = prediction[0]  # Aggiungi la previsione alla sequenza

# Inverse transform per ottenere le previsioni nella scala originale
future_predictions_original_scale = scaler.inverse_transform(np.array(future_predictions))

print("Future Predictions (Original Scale):")
print(future_predictions_original_scale)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
Future Predictions (Original Scale):
[[19.452148 28.876543 34.716557]
 [19.927769 29.606298 35.267162]
 [20.328224 30.123764 35.775127]
 [19.088512 28.159    33.640606]
 [19.443642 28.66832  34.09214 ]
 [19.906862 29.321451 34.813717]
 [19.469479 28.56513  34.069256]
 [18.942999 27.70698  33.091206]
 [19.224033 28.128937 33.56732 ]
 [17.942284 26.178326 31.547773]]


In [120]:
#noooooo

# prompt: dopo aver fatto il training calcola nuovamente la predizione in modo che sia sicuro dell'estrazione dei numeri

import numpy as np

# ... (preceding code)

# Previsione (modificata per maggiore sicurezza)
last_sequence = X[-1].reshape(1, sequence_length, X.shape[2])
predictions = []
for _ in range(5):  # Genera 5 previsioni per avere più sicurezza
    pred = model.predict(last_sequence)
    predicted_numbers = np.argsort(pred[0])[-5:] + 1
    predictions.append(predicted_numbers)
    # Aggiorna la sequenza per la prossima previsione (usando l'ultima previsione)
    next_step = pred.reshape(1, X.shape[2])
    last_sequence = np.concatenate((last_sequence[:, 1:, :], next_step.reshape(1,1,X.shape[2])), axis = 1)


# Calcola i numeri più frequenti nelle previsioni
from collections import Counter
all_predictions = np.concatenate(predictions)
predicted_numbers_freq = Counter(all_predictions)

# Seleziona i 5 numeri più frequenti
top_5_predicted = predicted_numbers_freq.most_common(5)
final_prediction = [num for num, freq in top_5_predicted]

print("Numeri Predetti (con maggiore sicurezza):", final_prediction)


ValueError: Exception encountered when calling LSTMCell.call().

[1mDimensions must be equal, but are 1 and 3 for '{{node sequential_9_1/lstm_11_1/lstm_cell_1/MatMul}} = MatMul[T=DT_FLOAT, grad_a=false, grad_b=false, transpose_a=false, transpose_b=false](sequential_9_1/lstm_11_1/strided_slice_1, sequential_9_1/lstm_11_1/lstm_cell_1/Cast/ReadVariableOp)' with input shapes: [1,1], [3,200].[0m

Arguments received by LSTMCell.call():
  • inputs=tf.Tensor(shape=(1, 1), dtype=float32)
  • states=('tf.Tensor(shape=(1, 50), dtype=float32)', 'tf.Tensor(shape=(1, 50), dtype=float32)')
  • training=False

In [121]:

import pandas as pd
import datetime

file_path = '/content/it-lotto-past-draws-archive.xls'

def carica_database(file_path):
    # Leggi il file Excel
    df = pd.read_excel(file_path, engine='xlrd', skiprows=1)
    # Prende solo numeri e rimuove colonne vuote
    numeri_estratti = df.select_dtypes(include='number').dropna(axis=1, how='all')

    # Usa solo le ultime 50 estrazioni 🔥
    numeri_estratti = numeri_estratti.tail(50)
    return numeri_estratti

def calcola_frequenze(df):
    frequenze = df.stack().value_counts()
    return frequenze.head(5).index.tolist()

def calcola_ritardatari(df):
    numeri_totali = set(range(1, 91))
    ultime_uscite = {}

    for numero in numeri_totali:
        try:
            ultima_pos = df.apply(lambda colonna: numero in colonna.values).iloc[::-1].idxmax()
            ultime_uscite[numero] = ultima_pos
        except:
            ultime_uscite[numero] = -1

    ritardatari = sorted(ultime_uscite, key=ultime_uscite.get, reverse=True)[:5]
    return ritardatari

def somma_90(previsione):
    somma = (previsione[0] + previsione[1]) % 90
    if somma == 0:
        somma = 90
    return somma

# Esegui tutto
df = carica_database(file_path)
numeri_frequenti = calcola_frequenze(df)
numeri_ritardatari = calcola_ritardatari(df)
previsione = numeri_frequenti + numeri_ritardatari
somma = somma_90(previsione)

print(f"Data aggiornamento: {datetime.date.today()}")
print("Numeri più frequenti:", numeri_frequenti)
print("Numeri ritardatari:", numeri_ritardatari)
print("Previsione Finale:", previsione)
print("🔥 Somma 90:", somma)



FileNotFoundError: [Errno 2] No such file or directory: '/content/it-lotto-past-draws-archive.xls'

In [122]:
# Genera una previsione (esempio semplificato)
# La previsione si basa sui numeri più frequenti
previsione = numeri_frequenti

NameError: name 'numeri_frequenti' is not defined

In [ ]:
# Crea un DataFrame per la previsione
previsione_df = pd.DataFrame({'Numero': previsione})


In [ ]:
# Salva la previsione in un file CSV
previsione_df.to_csv('/content/previsione_lotto.csv', index=False)

print("Previsione salvata in /content/previsione_lotto.csv")

from google.colab import files
files.download('/content/previsione_lotto.csv')


Previsione salvata in /content/previsione_lotto.csv


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>